In [ ]:
import nltk
nltk.download('punkt')
import pandas as pd

In [ ]:
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

In [ ]:
display(fake.info())

In [ ]:
display(true.info())

In [ ]:
display(fake.head(10))

In [ ]:
display(true.head(10))

In [ ]:
display(fake.subject.value_counts())
display(true.subject.value_counts())

In [ ]:
fake['flag'] = 0
true['flag'] = 1

In [ ]:
display(fake.flag.value_counts())
display(true.flag.value_counts())

In [ ]:
data = pd.concat([fake, true], axis=0)

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data = data.drop(['title', 'subject', 'date'], axis=1)

In [ ]:
data = pd.concat([fake, true], axis=0)

In [ ]:
print(data.columns)

# TOKENIZATION

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
data['text'] = data['text'].apply(word_tokenize)

In [ ]:
print(data.head(10))

# STEMMING

In [ ]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer("english")

In [ ]:
def stem_it(text):
    return [porter.stem(word) for word in text]

In [ ]:
data['text'] = data['text'].apply(stem_it)

In [ ]:
print(data.head(10))

# STOPWORD REMOVAL

In [ ]:
from nltk.corpus import stopwords

In [ ]:
def stop_it(t):
    dt = [word for word in t if len(word)>2]
    return dt

In [ ]:
data['text'] = data['text'].apply(stop_it)

In [ ]:
print(data.head(10))

In [ ]:
data['text'] = data['text'].apply(' '.join)

In [ ]:
print(data.head(10))

# SPLITTING

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['text'], data['flag'], test_size=0.25)

display(x_train.head())
print("\n")
display(y_train.head())

# VECTORIZATION

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf = TfidfVectorizer(max_df=0.7)

tfidf_train = my_tfidf.fit_transform(x_train)
tfidf_test = my_tfidf.transform(x_test)

In [ ]:
print(tfidf_train)

# LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model_1 = LogisticRegression(max_iter=900)
model_1.fit(tfidf_train, y_train)
predict_1 = model_1.predict(tfidf_test)
accur_pc = accuracy_score(y_test, predict_1)

In [ ]:
print(accur_pc*100)

# PASSIVE-AGGRESSIVE CLASSIFIER

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

model = PassiveAggressiveClassifier(max_iter=50)
model.fit(tfidf_train, y_train)

In [ ]:
y_predict = model.predict(tfidf_test)
accscore = accuracy_score(y_test, y_predict)
print(accscore*100)

In [ ]:
user_input = input("Enter a news article : ")
user_input = word_tokenize(user_input) 
user_input = stem_it(user_input)
user_input = stop_it(user_input)

In [ ]:
user_input = ' '.join(user_input)

In [ ]:
# Vectorize the user input
user_feature = my_tfidf.transform([user_input])

# Predict the category of the user input
predicted_category_1 = model_1.predict_proba(user_feature)

In [ ]:
print("Predicted category using Logistic Regression:", predicted_category_1)